In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

## Titanic
data_titanic = pd.read_csv("data/machine_failure/train.csv")

column_titanic_dict = {"PassengerId": "PassengerId", "Survived": "Survived", "Pclass": "TicketClass", "Name": "Name", "Sex": "Sex", "Age": "Age", "SibSp": "NumberSiblingsSpouses", "Parch": "NumberParentsChildren", "Ticket": "TicketNumber", "Fare": "Fare", "Cabin": "CabinNumber", "Embarked": "Port"}
data_titanic = data_titanic.rename(columns=column_titanic_dict)

train_X_titanic_columns = ["TicketClass", "Sex", "Age", "NumberSiblingsSpouses", "NumberParentsChildren", "Fare", "Port"]
train_y_titanic_columns = ["Survived"]
train_X_titanic_ordinal_columns = ["Sex", "Port"]

X_titanic = data_titanic[train_X_titanic_columns]
y_titanic = data_titanic[train_y_titanic_columns]
 
X_titanic.loc[:, train_X_titanic_ordinal_columns] = OrdinalEncoder().fit_transform(X_titanic[train_X_titanic_ordinal_columns])

X_titanic_train, X_titanic_test, y_titanic_train, y_titanic_test = train_test_split(X_titanic, y_titanic, test_size=0.4)
y_titanic_train = np.squeeze(y_titanic_train)
y_titanic_test = np.squeeze(y_titanic_test)

## Machine Failure
data_machine_failure = pd.read_csv("data/machine_failure/train.csv")

column_machine_failure_dict = {"PassengerId": "PassengerId", "Survived": "Survived", "Pclass": "TicketClass", "Name": "Name", "Sex": "Sex", "Age": "Age", "SibSp": "NumberSiblingsSpouses", "Parch": "NumberParentsChildren", "Ticket": "TicketNumber", "Fare": "Fare", "Cabin": "CabinNumber", "Embarked": "Port"}
data_machine_failure = data_machine_failure.rename(columns=column_machine_failure_dict)

train_X_machine_failure_columns = ["TicketClass", "Sex", "Age", "NumberSiblingsSpouses", "NumberParentsChildren", "Fare", "Port"]
train_y_machine_failure_columns = ["Survived"]
train_X_machine_failure_ordinal_columns = ["Sex", "Port"]

X_machine_failure = data_machine_failure[train_X_machine_failure_columns]
y_machine_failure = data_machine_failure[train_y_machine_failure_columns]
 
X_machine_failure.loc[:, train_X_machine_failure_ordinal_columns] = OrdinalEncoder().fit_transform(X_machine_failure[train_X_machine_failure_ordinal_columns])

X_machine_failure_train, X_machine_failure_test, y_machine_failure_train, y_machine_failure_test = train_test_split(X_machine_failure, y_machine_failure, test_size=0.4)
y_machine_failure_train = np.squeeze(y_machine_failure_train)
y_machine_failure_test = np.squeeze(y_machine_failure_test)

/var/folders/3f/d8gl3k3j0bn3wk_cqjvfm31w0000gn/T/ipykernel_77839/907899923.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_titanic.loc[:, train_X_titanic_ordinal_columns] = OrdinalEncoder().fit_transform(X_titanic[train_X_titanic_ordinal_columns])
/var/folders/3f/d8gl3k3j0bn3wk_cqjvfm31w0000gn/T/ipykernel_77839/907899923.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_machine_failure.loc[:, train_X_machine_failure_ordinal_columns] = OrdinalEncoder().fit_transform(X_machine_failure[train_X_mach

In [29]:
## Titanic
## Standard Scaler

import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, accuracy_score, recall_score

X_titanic_train_1, X_titanic_train_2, y_titanic_train_1, y_titanic_train_2 = train_test_split(X_titanic_train, y_titanic_train, test_size=0.5)
y_titanic_train_1 = np.squeeze(y_titanic_train_1)
y_titanic_train_2 = np.squeeze(y_titanic_train_2)
# Logistic Regression
param_grid = [
  {"logr_clf__penalty": ["l2"],
   "logr_clf__C": np.logspace(-1, 3, 40),
   "logr_clf__multi_class": ["auto", "ovr", "multinomial"]}
]
logr_clf = GridSearchCV(Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", StandardScaler()),
    ("logr_clf", LogisticRegression()),]),
    param_grid, cv=10, verbose=0)
logr_clf.fit(X_titanic_train_1, y_titanic_train_1)
# Ada Boost
param_grid = [
  {"ada_bst_clf__n_estimators": [100, 1000, 10000],
   "ada_bst_clf__learning_rate": [0.01, 0.1, 1]}
]
ada_bst_clf = GridSearchCV(Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", StandardScaler()),
    ("ada_bst_clf", AdaBoostClassifier(estimator=logr_clf.best_estimator_["logr_clf"])),]),
    param_grid, cv=10, verbose=3)
ada_bst_clf.fit(X_titanic_train_2, y_titanic_train_2)
# Predict
y_titanic_pred = ada_bst_clf.predict(X_titanic_test)
# Score
print("Confusion Matrix")
print(confusion_matrix(y_titanic_test, y_titanic_pred))
print("Precision Score")
print(precision_score(y_titanic_test, y_titanic_pred))
print("Accuracy Score")
print(accuracy_score(y_titanic_test, y_titanic_pred))
print("Recall Score")
print(recall_score(y_titanic_test, y_titanic_pred))
print("F1 Score")
print(f1_score(y_titanic_test, y_titanic_pred))

Fitting 10 folds for each of 9 candidates, totalling 90 fits
[CV 1/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.593 total time=   0.1s
[CV 2/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.593 total time=   0.1s
[CV 3/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.593 total time=   0.1s
[CV 4/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.593 total time=   0.1s
[CV 5/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.593 total time=   0.1s
[CV 6/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.593 total time=   0.1s
[CV 7/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.630 total time=   0.1s
[CV 8/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.615 total time=   0.1s
[CV 9/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf_

In [30]:
## Machine Failure
## Standard Scaler

import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, accuracy_score, recall_score


print("Machine Failure")
X_machine_failure_train_1, X_machine_failure_train_2, y_machine_failure_train_1, y_machine_failure_train_2 = train_test_split(X_machine_failure_train, y_machine_failure_train, test_size=0.5)
y_machine_failure_train_1 = np.squeeze(y_machine_failure_train_1)
y_machine_failure_train_2 = np.squeeze(y_machine_failure_train_2)
# Logistic Regression
param_grid = [
  {"logr_clf__penalty": ["l2"],
   "logr_clf__C": np.logspace(-1, 3, 40),
   "logr_clf__multi_class": ["auto", "ovr", "multinomial"]}
]
logr_clf = GridSearchCV(Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", StandardScaler()),
    ("logr_clf", LogisticRegression()),]),
    param_grid, cv=10, verbose=0)
logr_clf.fit(X_machine_failure_train_1, y_machine_failure_train_1)
# Ada Boost
param_grid = [
  {"ada_bst_clf__n_estimators": [100, 1000, 10000],
   "ada_bst_clf__learning_rate": [0.01, 0.1, 1]}
]
ada_bst_clf = GridSearchCV(Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", StandardScaler()),
    ("ada_bst_clf", AdaBoostClassifier(estimator=logr_clf.best_estimator_["logr_clf"])),]),
    param_grid, cv=10, verbose=3)
ada_bst_clf.fit(X_machine_failure_train_2, y_machine_failure_train_2)
# Predict
ada_bst_clf.fit(X_machine_failure_train, y_machine_failure_train)
y_machine_failure_pred = ada_bst_clf.predict(X_machine_failure_test)
# Score
print("Confusion Matrix")
print(confusion_matrix(y_machine_failure_test, y_machine_failure_pred))
print("Precision Score")
print(precision_score(y_machine_failure_test, y_machine_failure_pred))
print("Accuracy Score")
print(accuracy_score(y_machine_failure_test, y_machine_failure_pred))
print("Recall Score")
print(recall_score(y_machine_failure_test, y_machine_failure_pred))
print("F1 Score")
print(f1_score(y_machine_failure_test, y_machine_failure_pred))

Machine Failure
Fitting 10 folds for each of 9 candidates, totalling 90 fits
[CV 1/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.667 total time=   0.1s
[CV 2/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.667 total time=   0.1s
[CV 3/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.667 total time=   0.1s
[CV 4/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.667 total time=   0.1s
[CV 5/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.667 total time=   0.1s
[CV 6/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.630 total time=   0.1s
[CV 7/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.630 total time=   0.1s
[CV 8/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.654 total time=   0.1s
[CV 9/10] END ada_bst_clf__learning_rate=0.

In [31]:
## Titanic
## MinMiax Scaler

import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, accuracy_score, recall_score

X_titanic_train_1, X_titanic_train_2, y_titanic_train_1, y_titanic_train_2 = train_test_split(X_titanic_train, y_titanic_train, test_size=0.5)
y_titanic_train_1 = np.squeeze(y_titanic_train_1)
y_titanic_train_2 = np.squeeze(y_titanic_train_2)
# Logistic Regression
param_grid = [
  {"logr_clf__penalty": ["l2"],
   "logr_clf__C": np.logspace(-1, 3, 40),
   "logr_clf__multi_class": ["auto", "ovr", "multinomial"]}
]
logr_clf = GridSearchCV(Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", MinMaxScaler()),
    ("logr_clf", LogisticRegression()),]),
    param_grid, cv=10, verbose=0)
logr_clf.fit(X_titanic_train_1, y_titanic_train_1)
# Ada Boost
param_grid = [
  {"ada_bst_clf__n_estimators": [100, 1000, 10000],
   "ada_bst_clf__learning_rate": [0.01, 0.1, 1]}
]
ada_bst_clf = GridSearchCV(Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", MinMaxScaler()),
    ("ada_bst_clf", AdaBoostClassifier(estimator=logr_clf.best_estimator_["logr_clf"])),]),
    param_grid, cv=10, verbose=3)
ada_bst_clf.fit(X_titanic_train_2, y_titanic_train_2)
# Predict
y_titanic_pred = ada_bst_clf.predict(X_titanic_test)
# Score
print("Confusion Matrix")
print(confusion_matrix(y_titanic_test, y_titanic_pred))
print("Precision Score")
print(precision_score(y_titanic_test, y_titanic_pred))
print("Accuracy Score")
print(accuracy_score(y_titanic_test, y_titanic_pred))
print("Recall Score")
print(recall_score(y_titanic_test, y_titanic_pred))
print("F1 Score")
print(f1_score(y_titanic_test, y_titanic_pred))

Fitting 10 folds for each of 9 candidates, totalling 90 fits
[CV 1/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.630 total time=   0.1s
[CV 2/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.630 total time=   0.1s
[CV 3/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.630 total time=   0.1s
[CV 4/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.630 total time=   0.1s
[CV 5/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.630 total time=   0.1s
[CV 6/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.593 total time=   0.1s
[CV 7/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.593 total time=   0.1s
[CV 8/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.615 total time=   0.1s
[CV 9/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf_

In [32]:
## Machine Failure
## MinMax Scaler

import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, accuracy_score, recall_score


print("Machine Failure")
X_machine_failure_train_1, X_machine_failure_train_2, y_machine_failure_train_1, y_machine_failure_train_2 = train_test_split(X_machine_failure_train, y_machine_failure_train, test_size=0.5)
y_machine_failure_train_1 = np.squeeze(y_machine_failure_train_1)
y_machine_failure_train_2 = np.squeeze(y_machine_failure_train_2)
# Logistic Regression
param_grid = [
  {"logr_clf__penalty": ["l2"],
   "logr_clf__C": np.logspace(-1, 3, 40),
   "logr_clf__multi_class": ["auto", "ovr", "multinomial"]}
]
logr_clf = GridSearchCV(Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", MinMaxScaler()),
    ("logr_clf", LogisticRegression()),]),
    param_grid, cv=10, verbose=0)
logr_clf.fit(X_machine_failure_train_1, y_machine_failure_train_1)
# Ada Boost
param_grid = [
  {"ada_bst_clf__n_estimators": [100, 1000, 10000],
   "ada_bst_clf__learning_rate": [0.01, 0.1, 1]}
]
ada_bst_clf = GridSearchCV(Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", MinMaxScaler()),
    ("ada_bst_clf", AdaBoostClassifier(estimator=logr_clf.best_estimator_["logr_clf"])),]),
    param_grid, cv=10, verbose=3)
ada_bst_clf.fit(X_machine_failure_train_2, y_machine_failure_train_2)
# Predict
ada_bst_clf.fit(X_machine_failure_train, y_machine_failure_train)
y_machine_failure_pred = ada_bst_clf.predict(X_machine_failure_test)
# Score
print("Confusion Matrix")
print(confusion_matrix(y_machine_failure_test, y_machine_failure_pred))
print("Precision Score")
print(precision_score(y_machine_failure_test, y_machine_failure_pred))
print("Accuracy Score")
print(accuracy_score(y_machine_failure_test, y_machine_failure_pred))
print("Recall Score")
print(recall_score(y_machine_failure_test, y_machine_failure_pred))
print("F1 Score")
print(f1_score(y_machine_failure_test, y_machine_failure_pred))

Machine Failure
Fitting 10 folds for each of 9 candidates, totalling 90 fits
[CV 1/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.778 total time=   0.1s
[CV 2/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.889 total time=   0.1s
[CV 3/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.667 total time=   0.1s
[CV 4/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.889 total time=   0.1s
[CV 5/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.815 total time=   0.1s
[CV 6/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.852 total time=   0.1s
[CV 7/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.593 total time=   0.1s
[CV 8/10] END ada_bst_clf__learning_rate=0.01, ada_bst_clf__n_estimators=100;, score=0.769 total time=   0.1s
[CV 9/10] END ada_bst_clf__learning_rate=0.